## Synchronised Chaos

### Implementing the transmitter and receiver

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# Parameters of a chaotic Lorenz system
r, s, b = (28, 10, 8/3)

# Setting the time step and total simulation time
dt = 0.001
steps = 60000

# Initial conditions for transmitter
state_transmitter = np.empty((steps + 1, 3))  # Need one more for the initial values
state_transmitter[0] = (0, 1.0, 1.05) 

# Initial conditions for receiver
state_receiver = np.empty((steps + 1, 3)) 
state_receiver[0] = (0.1, 3.0, 0.05)  # If the system synchronises, any initial values work

t = np.linspace(0, steps*dt, steps+1)   # time array

# Message signal sample
message = 5*np.sin(60*t)

# Defining the transmitter Lorenz system
def transmitter(initial_transmitter):
    x, y, z = initial_transmitter   # Initial conditions unpacked as a tuple
    dx_dt = s*(y-x)
    dy_dt = r*x - y - x*z
    dz_dt = x*y - b*z
    return np.array([dx_dt, dy_dt, dz_dt])

# Defining the receiver Lorenz system
def receiver(initial_receiver, input_signal):
    x, y, z = initial_receiver 
    dx_dt = s*(y-x) 
    dy_dt = r*input_signal - y - input_signal*z
    dz_dt = input_signal*y - b*z
    return np.array([dx_dt, dy_dt, dz_dt])

# 4th Order Runge Kutta method (RK4)
def RK4(dt, function, state, *args):
    k1 = function(state, *args)
    k2 = function(state + (dt*k1)/2, *args)
    k3 = function(state + (dt*k2)/2, *args)
    k4 = function(state + dt*k3, *args)
    return state + (dt/6)*(k1 + 2*k2 + 2*k3 + k4)

# Use RK4 to numerically solve for transmitter system
for i in range(steps):
    state_transmitter[i + 1] = RK4(dt, transmitter, state_transmitter[i])

# Generating the mask to encrypt the signal
x_values = state_transmitter[:, 0]  
masked_values = x_values + message

# Use RK4 to numerically solve for receiver system
for i in range(steps):
    state_receiver[i + 1] = RK4(dt, receiver, state_receiver[i], masked_values[i])

# Regenerating the mask to decrypt the signal
xr_values = state_receiver[:, 0]
decrypt_values = masked_values - xr_values
error = np.abs(decrypt_values - message) # Error should decrease exponentially with time

# Calculate RMSE in decryption
def RMSE(error):
    return np.sqrt(np.mean(error**2))

# "Normalise" the RMSE relative to amplitude
rms_error = RMSE(error) / (np.max(message) - np.min(message))
print(rms_error)

In [ ]:
# For observing a range of interest
start_view = 12000
end_view = 20000
t_view = t[start_view:end_view]
m_view = message[start_view:end_view]
x_view = x_values[start_view:end_view]
masked_view = masked_values[start_view:end_view]
decrypt_view = decrypt_values[start_view:end_view]

# Plotting the chaotic mask and message signal
ax = plt.figure().add_subplot()
ax.plot(t_view, x_view)
ax.set_xlabel("Time step")
ax.set_ylabel("Masking Values")
ax.set_title('Masking values vs Time')
plt.scatter(t_view, m_view, c="orange", s=1)

# Plotting the masked signal
ax = plt.figure().add_subplot()
ax.plot(t_view, masked_view)
ax.set_xlabel("Time step")
ax.set_ylabel("Masked Values")
ax.set_title('Masked values vs Time')

# Plotting the retrieved signal
ax = plt.figure().add_subplot()
ax.plot(t_view, decrypt_view)
ax.set_xlabel("Time step")
ax.set_ylabel("Decrypted Values")
ax.set_title('Decrypted Values vs Time')
plt.scatter(t_view, m_view, c="orange", s=1)

# Plotting the error
ax = plt.figure().add_subplot()
ax.plot(t, error)
ax.set_xlabel("Time step")
ax.set_ylabel("Error Values")
ax.set_title('Error between receiver and transmitter')

### Processing audio files

In [ ]:
from scipy.io.wavfile import read
from scipy.io.wavfile import write

In [ ]:
# Read a WAV file
sample_rate, data = read("gameover.wav")
print(f"The sample rate is {sample_rate} Hz")
print(f"There are {len(data)} samples, filetype is {data.dtype} bits")   # Number of timesteps need to match number of samples

In [ ]:
# Processing a WAV file

simulation_time = steps * dt
amplitude = 2**15   # To normalise the amplitude for a 16 bit file
time = np.arange(len(data)) / sample_rate  # Time axis in seconds
time_scale = simulation_time / time[steps+1]
times = time_scale * time[:steps+1]   # Converting to time steps
sound_waves = (1/amplitude)*data[:steps+1,0]   # Using the left stereo

plt.figure(figsize=(14, 4))
plt.plot(time[:steps+1], sound_waves)
plt.xlabel("Time/steps")
plt.ylabel("Amplitude")
plt.title("Waveform")

# Encrpyting the audio file
masked_sound = x_values + sound_waves

# Use RK4 to numerically solve for receiver system
for i in range(steps):
    state_receiver[i + 1] = RK4(dt, receiver, state_receiver[i], masked_sound[i])

# Regenerating the mask to decrypt the signal
xs_values = state_receiver[:, 0]
decrypt_sound = masked_sound - xs_values
error_sound = np.abs(decrypt_sound - sound_waves)

# Calculate RMSE in decryption
rms_error_sound = np.sqrt(np.mean((error_sound**2)))
print(rms_error_sound)

# Plotting the chaotic mask and message signal
ax = plt.figure().add_subplot()
ax.plot(t, x_values)
ax.set_xlabel("Time step")
ax.set_ylabel("Masking Values")
ax.set_title('Masking values vs Time')
plt.scatter(times, sound_waves, c="orange", s=1)

# Plotting the masked signal
ax = plt.figure().add_subplot()
ax.plot(t, masked_sound)
ax.set_xlabel("Time step")
ax.set_ylabel("Masked Values")
ax.set_title('Masked values vs Time')

# Plotting the retrieved signal
ax = plt.figure().add_subplot()
ax.plot(t, decrypt_sound)
ax.set_xlabel("Time step")
ax.set_ylabel("Decrypted Values")
ax.set_title('Decrypted Values vs Time')
plt.scatter(times, sound_waves, c="orange", s=1)

# Plotting the error
ax = plt.figure().add_subplot()
ax.plot(t, error)
ax.set_xlabel("Time step")
ax.set_ylabel("Error Values")
ax.set_title('Error between receiver and transmitter')

plt.tight_layout()
plt.show()

In [ ]:
# Writing the output file
y1_int16 = (masked_sound * amplitude).astype(np.int16)
y2_int16 = (decrypt_sound * amplitude).astype(np.int16)

write("encrypted.wav", sample_rate, y1_int16)
write("decrypted.wav", sample_rate, y2_int16)

# Image encryption Trial

In [2]:
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

im=Image.open(r'C:\Undergraduate_NUS\Semester 1\SP2273\Lorenz encryption\SP2273_Group_project_Chaos_Sync\97-973734_instagram-clipart-picsart-png-instagram-logo-100x100-png.jpg')
def unwrapper(image):
    img_array=np.asarray(image)
    #flattening the array
    return img_array.flatten()

#Stringing everything together
#now the flattened array is basically (rgb_rgb_rgb.....rgb)*width*height
def wrapper(ar1,height,width):
    #my array has shape (height,width,3)
    return Image.fromarray(ar1.reshape(height,width,3),'RGB')
# Parameters of a chaotic Lorenz system
r, s, b = (28, 10, 8/3)

# Setting the time step and total simulation time
dt = 0.001
steps = im.size[1]*im.size[0]*3-1

# Initial conditions for transmitter
state_transmitter = np.empty((steps + 1, 3))  # Need one more for the initial values
state_transmitter[0] = (0, 1.0, 1.05) 

# Initial conditions for receiver
state_receiver = np.empty((steps + 1, 3)) 
state_receiver[0] = (0.1, 3.0, 0.05)  # If the system synchronises, any initial values work

t = np.linspace(0, steps*dt, steps+1)   # time array

# Message signal sample
a = 1.0   # Modulate amplitude, if needed
'''m = np.sin(60*t) + np.cos(40*t) + np.sin(20*np.pi*t) + np.cos(30*t)'''
m=unwrapper(im)

# Defining the transmitter Lorenz system
def transmitter(initial_transmitter):
    x, y, z = initial_transmitter   # Initial conditions unpacked as a tuple
    dx_dt = s*(y-x)
    dy_dt = r*x - y - x*z
    dz_dt = x*y - b*z
    return np.array([dx_dt, dy_dt, dz_dt])

# Defining the receiver Lorenz system
def receiver(initial_receiver, input_signal):
    x, y, z = initial_receiver 
    dx_dt = s*(y-x) 
    dy_dt = r*input_signal - y - input_signal*z
    dz_dt = input_signal*y - b*z
    return np.array([dx_dt, dy_dt, dz_dt])

# 4th Order Runge Kutta method (RK4)
def RK4(dt, function, state, *args):
    k1 = function(state, *args)
    k2 = function(state + (dt*k1)/2, *args)
    k3 = function(state + (dt*k2)/2, *args)
    k4 = function(state + dt*k3, *args)
    return state + (dt/6)*(k1 + 2*k2 + 2*k3 + k4)

# Use RK4 to numerically solve for transmitter system
for i in range(steps):
    state_transmitter[i + 1] = RK4(dt, transmitter, state_transmitter[i])

# Generating the mask to encrypt the signal
x_values = state_transmitter[:, 0]  
masked_values = x_values + m

# Use RK4 to numerically solve for receiver system
for i in range(steps):
    state_receiver[i + 1] = RK4(dt, receiver, state_receiver[i], masked_values[i])

# Regenerating the mask to decrypt the signal
xr_values = state_receiver[:, 0]
decrypt = masked_values - xr_values
error = np.abs(xr_values - x_values) # Error should decrease exponentially with time
image=wrapper(xr_values,im.size[1],im.size[0])
image.save("output_zs_image.jpg", "jpeg")
image1=wrapper(unwrapper(im),im.size[1],im.size[0])
image1.save("test_output_zs_image.jpg", "jpeg")

C:\Users\User\AppData\Local\Temp\ipykernel_14060\3825852945.py:15: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  return Image.fromarray(ar1.reshape(height,width,3),'RGB')


## From Zs

### Investigating inputs
- Amplitude (Need to reconcile with theory)
- Length of simulation (work in progress)
- Frequency of sin function (graph needs explanation)
- Different test functions (output needs explanation)

Current issues: 
For amplitude, RMSE alone might not be meaningful as smaller message amplitude results in smaller RMSE values but graphically depicts a poorer fit to the decryted signal.

For length of simulation, the code works for an initial run but subsequent runs will output constant valus. The transmitter and receiver implementation code needs to be run for the code to work again.

No idea why the graphs for RMSE vs frequency and the respective input functions behave these ways.

In [ ]:
# Amplitude
amp_errors = []
for i in range (1, 100): 
    amp_function = (i/50)*np.sin(60*t)
    mask_test_amp = amp_function + x_values
    for j in range(int(steps/10)):
        state_receiver[j + 1] = RK4(dt, receiver, state_receiver[j], mask_test_amp[j])
    xr_values_amp = state_receiver[:, 0]
    decrypt_test_amp = mask_test_amp - xr_values_amp
    error_amp = np.abs(decrypt_test_amp - amp_function)
    rms_error_amp = RMSE(error_amp)/(np.max(amp_function) - np.min(amp_function))
    amp_errors.append(rms_error_amp)

print(amp_errors)
amp = [i/50 for i in range(len(amp_errors))]

ax = plt.figure().add_subplot()
ax.plot(amp, amp_errors)
ax.set_xlabel("Amplitudes")
ax.set_ylabel("RMSE values")
ax.set_title('RMSE values vs Amplitudes')

In [ ]:
# Changing length of simulation

function_test_length = np.sin(60*t)
mask_test_length = function_test_length + x_values
length_errors = []
for i in range (10, steps, 2000): # To-do: use more steps. Here for 16 data point it takes 16s.
    for j in range(i):
        state_receiver[j + 1] = RK4(dt, receiver, state_receiver[j], mask_test_length[j])
    xr_values_length = state_receiver[:, 0]
    decrypt_test_length = mask_test_length - xr_values_length
    error_length = np.abs(decrypt_test_length - function_test_length)
    rms_error_length = RMSE(error_length)
    length_errors.append(rms_error_length)

lengths = [i for i in range(len(length_errors))]

ax = plt.figure().add_subplot()
ax.plot(lengths, length_errors)
ax.set_xlabel("Length of simulation")
ax.set_ylabel("RMSE values")
ax.set_title('RMSE values vs Simulation time')

In [ ]:
# Frequency
freq_errors = []
for i in range (1, 100): # Takes 25s with 0.001 dt and 10000 steps
    freq_function = np.sin(i*t)
    mask_test_freq = freq_function + x_values
    for j in range(int(steps/10)):
        state_receiver[j + 1] = RK4(dt, receiver, state_receiver[j], mask_test_freq[j])
    xr_values_freq = state_receiver[:, 0]
    decrypt_test_freq = mask_test_freq - xr_values_freq
    error_freq = np.abs(decrypt_test_freq - freq_function)
    rms_error_freq = RMSE(error_freq)
    freq_errors.append(rms_error_freq)

freq = [i for i in range(len(freq_errors))]

ax = plt.figure().add_subplot()
ax.plot(freq, freq_errors)
ax.set_xlabel("Frequencies")
ax.set_ylabel("RMSE values")
ax.set_title('RMSE values vs Frequency')

In [ ]:
# Fourier series
square_wave = 0
for i in range(1, 2000, 2):
    square_wave += (np.sin(60*np.pi*i*t))/i
square_wave *= 4/np.pi

sawtooth_wave = 0
for i in range(1, 2000):
    sawtooth_wave += ((-1)**i)*(np.sin(60*np.pi*i*t))/i
sawtooth_wave *= 2/np.pi

# Non-periodic functions
non_periodic_1 = np.sin(np.exp(t))   # Frequency increases as t increases
non_periodic_2 = np.cos(30*np.pi*np.sqrt(t))   # Frequency decreases as t increases

# Generating a random array of values [-1, 1]
random_function = 2 * np.random.rand(steps+1) - 1

# Function for control and comparison
sin_function = np.sin(60*t)

func_list = [square_wave, sawtooth_wave, non_periodic_1, 
                  non_periodic_2, random_function, sin_function]

func_namelist = ["Square Wave", "Sawtooth Wave", r"\sin(e^x)", 
                  r"\cos(\pi*\sqrt{x})", "Random Function", "Sin function"]

func_masked_list = [func + x_values for func in func_list ]

func_decrypt_list = []

func_error_list = []

for index, func_masked in enumerate(func_masked_list):
    for i in range(steps):
        state_receiver[i + 1] = RK4(dt, receiver, state_receiver[i], func_masked[i])
    xr_func = state_receiver[:, 0]
    decrypt_func = func_masked - xr_func
    func_error = np.abs(decrypt_func - func_list[index])
    func_rms_error = RMSE(func_error)

    # Append respective data to corresponding lists
    func_decrypt_list.append(decrypt_func)
    func_error_list.append(func_rms_error)

for index, func in enumerate(func_namelist):
    print(f"The RMSE for {func} is {func_error_list[index]}")

for index, func in enumerate(func_namelist):
    ax = plt.figure().add_subplot()
    ax.plot(t_view, func_decrypt_list[index][start_view:end_view])
    ax.set_xlabel("Time step")
    ax.set_ylabel("Decrypted Values")
    ax.set_title(f"${func}$")
    plt.scatter(t_view, func_list[index][start_view:end_view], c="orange", s=1)

